In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.preprocessing as pp
import utils.io_model as im
import utils.join_race_data as join_race_data
import utils.prepare_data as prepare_data
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path
import tensorflow as tf

In [ ]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

In [ ]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
TARGET_DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/test_data/arima'
TRAIN_DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/train_data'

# データ読み込み・前処理

In [ ]:
target_horse_df = rd.read_target_horse_csv(TARGET_DATA_PATH)
target_race_df  = rd.read_target_race_csv(TARGET_DATA_PATH)
target_df = pd.merge(target_horse_df, target_race_df, on='race_id', how='left')

In [ ]:
# ターゲットデータに過去3レース分の情報を追加
past_data_df = rd.read_horse_race_csv(TRAIN_DATA_PATH)
columns_past_data = [c for c in past_data_df.columns if "-" not in c]
df_for_prediction = join_race_data.join_n_race_for_test_data(past_data_df[columns_past_data], target_df, 3)

In [ ]:
df_for_prediction.head()

In [ ]:
df_for_prediction = prepare_data.prepare_data_for_prediction(df_for_prediction, past_data_df)

In [ ]:
df_for_prediction.head()

# 予測

In [ ]:
model = im.read_model("second_model")

In [ ]:
x = np.array(df_for_prediction)

In [ ]:
test = []
for x_input, umaban, horse_id in zip(x, list(range(len(target_df["horse_id"]))), list(target_df["horse_id"])):
    pred = model.predict(x_input.reshape(1, -1))
    if umaban == 1 or umaban == 2 or umaban == 4 or umaban == 5:
        test.append(x_input)
    prediction_class = np.argmax([pred[0][0], pred[0][1], pred[0][2]])
    print("class: %d, (%f, %f, %f), %d (%d)" % (prediction_class, pred[0][0], pred[0][1], pred[0][2], umaban + 1, horse_id))